In [109]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

#tb.loc[tb['Instrument Classification (ISO 10962 CFI code)'].isin(['JFTXFP'])]
#tb.plot.hist(tb.groupby(['Trading platform']).Price.agg(['mean', max]))
#tb.describe()
#print(tb.shape)

#df = pd.read_feather('C:/Users/asus/Documents/rts27.feather')
df = pd.read_feather('C:/Users/inald/Documents/NWMprod/wetransfer-0f4d76/rts27.feather')

#Columns deemed unneccessary for this task
df2 = df.drop(['file_name','designated_market_makers','legal_entity_name', 'systematic_internaliser_flag'], axis = 1)

df2.head(10)

,id,bank_name,date_of_trading_day,instrument_identifier,instrument_classification,order_request_quotes_received,executed_transactions,total_value_of_transactions,order_request_quotes_can_wdrwn,order_request_quotes_modified,...,median_transaction_size_eur,median_transaction_size_gbp,median_size_request_quotes_eur,median_size_request_quotes_gbp,smartie_instrument_name,data_quality_flag,egb_eligible_flag,year_quarter,business,central_bank_isin_flag
0,77854033,UBS,2018-12-18,CH0386503236,None,0.0,1.0,20.000,0.0,0.0,...,20.000000,17.981601,NaN,NaN,None,Y,None,2018 Q4,None,None
1,77854248,UBS,2018-12-20,CH0386504416,None,0.0,2.0,400.000,0.0,0.0,...,200.000000,180.523989,NaN,NaN,None,Y,None,2018 Q4,None,None
2,77856619,UBS,2018-10-08,CH0394745357,None,0.0,1.0,100.000,0.0,0.0,...,100.000000,87.869988,NaN,NaN,None,Y,None,2018 Q4,None,None
3,77818775,UBS,2018-11-27,AN8068571086,None,0.0,2.0,3184.000,0.0,0.0,...,2723.487198,2413.062250,NaN,NaN,None,Y,None,2018 Q4,None,None
4,77818788,UBS,2018-10-31,AN8068571086,None,0.0,4.0,77641.000,0.0,0.0,...,963.780919,854.948672,NaN,NaN,None,Y,None,2018 Q4,None,None
5,78737782,UBS,2018-12-07,EZKDQ6DRGH95,None,0.0,1.0,8904.901,0.0,0.0,...,7816.011236,6983.529412,NaN,NaN,None,Y,None,2018 Q4,None,None
6,78738980,UBS,2018-11-19,EZKGZGMRWNG1,None,0.0,5.0,3598.310,0.0,0.0,...,268.972142,239.539586,NaN,NaN,None,Y,None,2018 Q4,None,None
7,78744698,UBS,2018-11-30,EZKSTQRK3284,None,0.0,2.0,3433.450,0.0,0.0,...,1770.474424,1570.163755,NaN,NaN,None,Y,None,2018 Q4,None,None
8,78744713,UBS,2018-12-21,EZKSVCDDP7S3,None,0.0,2.0,61202.440,0.0,0.0,...,33552.908199,30220.118343,NaN,NaN,None,Y,None,2018 Q4,None,None
9,78744728,UBS,2018-12-28,EZKSVJYKDXB9,None,0.0,1.0,6697.360,0.0,0.0,...,5852.486236,5276.552159,NaN,NaN,None,Y,None,2018 Q4,None,None


In [111]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import datasets, linear_model
from sklearn.decomposition import TruncatedSVD


from datetime import datetime

#Columns deemed unneccessary for this task
#df = df.drop(['file_name','designated_market_makers','legal_entity_name', 'systematic_internaliser_flag'], axis = 1)
df = df2


#Filters the code to FX forwards only
#df = df.loc[(df['source_instrument_desc']).astype(str).str.contains("Forward")]
df = df.loc[(df['instrument_classification']).astype(str).str.contains("JF") | (df['source_instrument_desc']).astype(str).str.contains("Forward")]
#Drops the columns of which half of the fields have no data.
#df = df.dropna(axis=1, thresh=2000000)

        
# Select target
y = df['date_created']
# To keep things simple, we'll use only numerical predictors
dropped_df = df.drop(['date_created', 'instrument_identifier'], axis=1)
X = dropped_df


# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

categorical_cols = [col for col in X.columns if X[col].dtype in ['object']]

# Select numerical columns
numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train[my_cols].copy()
X_valid = X_valid[my_cols].copy()

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant',missing_values=np.nan, fill_value=0)

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', missing_values= None, fill_value='missing_value')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

sc = StandardScaler()

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

model = linear_model.LinearRegression()

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('scale', StandardScaler(with_mean=False)),
                              ('reduce_dims', TruncatedSVD(n_components=5)),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Plot outputs
print(preds)
#plt.plot(X_valid, preds, color='blue', linewidth=3)

[1.55364000e+18 1.55365678e+18 1.54387261e+18 ... 1.54368773e+18
 1.55271104e+18 1.54806028e+18]


In [120]:
import unittest
import os

TESTDATA_FILENAME = 'C:/Users/inald/Documents/NWMprod/wetransfer-0f4d76/rts27.feather'
dfTest = pd.read_feather('C:/Users/inald/Documents/NWMprod/wetransfer-0f4d76/rts27.feather')

class TestNotebook(unittest.TestCase):
    
    
    def test_add(self):
        self.assertEqual(4,4)

    #checks that the dataframes contain frames
    def test_data_read(self):
        if dfTest.empty:
            self.assertTrue(False)
        else:
            self.assertTrue(True)
    
    #Test to check that columns are disposed of that are not needed        
    def test_dropping_columns(self):
        dropped_df = dfTest.drop(['date_created', 'instrument_identifier'], axis=1)
        Xtest = dropped_df
        count = 0
        count2 = 0
        for col in dfTest.columns:
            count = count + 1
        for col in Xtest.columns:
            count2 = count + 1
        self.assertNotEqual(count, count2)
        
   #Test to check that the data is split properly - test the 80% v 20% difference    
    def test_splitting_data_size(self):
        Xtest = dfTest.drop(['date_created', 'instrument_identifier'], axis=1)
        ytest = dfTest['date_created']
        Xtest_train, Xtest_valid, ytest_train, ytest_valid = train_test_split(Xtest, ytest, train_size=0.8, test_size=0.2, random_state=0)
        if Xtest_train.size > ytest_train.size:
            self.assertTrue(True)
        else:
            self.assertTrue(False)
    
    unittest.main(argv=[''], verbosity=3, exit=False)
   

test_add (__main__.TestNotebook) ... ok
test_data_read (__main__.TestNotebook) ... ok
test_dropping_columns (__main__.TestNotebook) ... ok
test_splitting_data_size (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 4 tests in 297.392s

OK
